<a href="https://colab.research.google.com/github/Ayushomega14/Ayushomega14/blob/main/Stock_Market_Trend_Prediction_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.1/985.1 kB 6.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import statsmodels.api as sm
from arch import arch_model
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import xgboost as xgb

from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/bajaj_stock_data.csv'
df = pd.read_csv(file_path)

# Ensure the Date column is in the correct format and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Drop NaN values
df = df.dropna()

# Scale the features 'Close', 'Open', 'High', 'Low', and 'Volume'
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['Close', 'Open', 'High', 'Low', 'Volume']])

# Create datasets with multiple features (using 'Close', 'Open', 'High', 'Low', 'Volume')
def create_dataset_multifeature(dataset, look_back=120):
    X, y = [], []
    for i in range(look_back, len(dataset)):
        X.append(dataset[i-look_back:i])  # Include all features
        y.append(dataset[i, 0])  # Predict 'Close' price (index 0)
    return np.array(X), np.array(y)

# Increase the look-back window to capture longer trends
look_back = 120
X, y = create_dataset_multifeature(scaled_data, look_back)

# Reshape X to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Build the LSTM model with increased complexity
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.3))  # Adjust dropout for regularization
model.add(LSTM(units=100, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(units=50))  # Extra Dense layer for more complexity
model.add(Dense(units=1))  # Output layer

# Compile the model with a lower learning rate
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Predictions and inverse transformation
predictions = model.predict(X_test)

# Inverse transformation for predictions
predictions = scaler.inverse_transform(
    np.concatenate((predictions, np.zeros((predictions.shape[0], X_test.shape[2]-1))), axis=1)
)[:, 0]

# Inverse transform y_test for comparison
y_test_scaled = scaler.inverse_transform(
    np.concatenate((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], X_test.shape[2]-1))), axis=1)
)[:, 0]

# --- ARIMA Model ---
arima_order = (5, 1, 0)  # (p, d, q)
arima_model = sm.tsa.ARIMA(df['Close'][:len(X_train) + look_back], order=arima_order)
arima_fitted = arima_model.fit()
arima_predictions = arima_fitted.forecast(steps=len(X_test))

# Convert ARIMA predictions from Series to NumPy array for scaling
arima_predictions_np = np.array(arima_predictions).reshape(-1, 1)

# Inverse scale the ARIMA predictions
arima_predictions_scaled = scaler.inverse_transform(
    np.concatenate((arima_predictions_np, np.zeros((arima_predictions_np.shape[0], X_test.shape[2]-1))), axis=1)
)[:, 0]

# --- GARCH Model ---
garch_model = arch_model(df['Close'][:len(X_train) + look_back], vol='Garch', p=1, q=1)
garch_fitted = garch_model.fit(disp="off")
garch_forecast = garch_fitted.forecast(horizon=len(X_test))
garch_predictions = garch_forecast.mean.values[-1, :]

# Inverse scale the GARCH predictions
garch_predictions_scaled = scaler.inverse_transform(
    np.concatenate((garch_predictions.reshape(-1, 1), np.zeros((garch_predictions.shape[0], X_test.shape[2]-1))), axis=1)
)[:, 0]

# --- Exponential Smoothing (ETS) Model ---
ets_model = ExponentialSmoothing(df['Close'][:len(X_train) + look_back], trend='add', seasonal='add', seasonal_periods=12)
ets_fitted = ets_model.fit()
ets_predictions = ets_fitted.forecast(steps=len(X_test))

# Convert ETS predictions to NumPy array for scaling
ets_predictions_np = np.array(ets_predictions).reshape(-1, 1)

# Inverse scale the ETS predictions
ets_predictions_scaled = scaler.inverse_transform(
    np.concatenate((ets_predictions_np, np.zeros((ets_predictions_np.shape[0], X_test.shape[2]-1))), axis=1)
)[:, 0]

# --- XGBoost Model ---
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1)
xgb_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)
xgb_predictions = xgb_model.predict(X_test.reshape(X_test.shape[0], -1))

# Inverse scale the XGBoost predictions
xgb_predictions_scaled = scaler.inverse_transform(
    np.concatenate((xgb_predictions.reshape(-1, 1), np.zeros((xgb_predictions.shape[0], X_test.shape[2]-1))), axis=1)
)[:, 0]

# Plot each model against LSTM
plt.figure(figsize=(16, 8))

# --- ARIMA vs LSTM ---
plt.subplot(5, 1, 1)
plt.plot(df.index[-len(y_test):], y_test_scaled, label='Actual Price', color='orange')
plt.plot(df.index[-len(y_test):], predictions, label='LSTM Predictions', color='green', linewidth=2)
plt.plot(df.index[-len(y_test):], arima_predictions_scaled, label='ARIMA Predictions', color='red')
plt.title('ARIMA vs LSTM')
plt.legend()

# --- GARCH vs LSTM ---
plt.subplot(5, 1, 2)
plt.plot(df.index[-len(y_test):], y_test_scaled, label='Actual Price', color='orange')
plt.plot(df.index[-len(y_test):], predictions, label='LSTM Predictions', color='green', linewidth=2)
plt.plot(df.index[-len(y_test):], garch_predictions_scaled, label='GARCH Predictions', color='blue')
plt.title('GARCH vs LSTM')
plt.legend()

# --- ETS vs LSTM ---
plt.subplot(5, 1, 3)
plt.plot(df.index[-len(y_test):], y_test_scaled, label='Actual Price', color='orange')
plt.plot(df.index[-len(y_test):], predictions, label='LSTM Predictions', color='green', linewidth=2)
plt.plot(df.index[-len(y_test):], ets_predictions_scaled, label='ETS Predictions', color='purple')
plt.title('ETS vs LSTM')
plt.legend()

# --- XGBoost vs LSTM ---
plt.subplot(5, 1, 4)
plt.plot(df.index[-len(y_test):], y_test_scaled, label='Actual Price', color='orange')
plt.plot(df.index[-len(y_test):], predictions, label='LSTM Predictions', color='green', linewidth=2)
plt.plot(df.index[-len(y_test):], xgb_predictions_scaled, label='XGBoost Predictions', color='brown')
plt.title('XGBoost vs LSTM')
plt.legend()

# --- All models together ---
plt.subplot(5, 1, 5)
plt.plot(df.index[-len(y_test):], y_test_scaled, label='Actual Price', color='orange')
plt.plot(df.index[-len(y_test):], predictions, label='LSTM Predictions', color='green', linewidth=2)
plt.plot(df.index[-len(y_test):], arima_predictions_scaled, label='ARIMA Predictions', color='red')
plt.plot(df.index[-len(y_test):], garch_predictions_scaled, label='GARCH Predictions', color='blue')
plt.plot(df.index[-len(y_test):], ets_predictions_scaled, label='ETS Predictions', color='purple')
plt.plot(df.index[-len(y_test):], xgb_predictions_scaled, label='XGBoost Predictions', color='brown')
plt.title('All Models vs LSTM')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluation metrics for each model
models = {'ARIMA': arima_predictions_scaled, 'GARCH': garch_predictions_scaled, 'ETS': ets_predictions_scaled, 'XGBoost': xgb_predictions_scaled}
for model_name, model_preds in models.items():
    mse = mean_squared_error(y_test_scaled, model_preds)
    mae = mean_absolute_error(y_test_scaled, model_preds)
    print(f"{model_name} - MSE: {mse}, MAE: {mae}")

# LSTM evaluation already computed
print(f"LSTM - MSE: {mean_squared_error(y_test_scaled, predictions)}, MAE: {mean_absolute_error(y_test_scaled, predictions)}")

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 29s 226ms/step - loss: 1.7908e-04 - mae: 0.0076 - val_loss: 4.0077e-04 - val_mae: 0.0131
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 45s 264ms/step - loss: 1.2683e-05 - mae: 0.0022 - val_loss: 7.0875e-04 - val_mae: 0.0189
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 35s 208ms/step - loss: 1.2361e-05 - mae: 0.0022 - val_loss: 3.2713e-04 - val_mae: 0.0101
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 43s 226ms/step - loss: 1.3772e-05 - mae: 0.0022 - val_loss: 4.7480e-04 - val_mae: 0.0141
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 38s 195ms/step - loss: 1.3837e-05 - mae: 0.0024 - val_loss: 8.2071e-04 - val_mae: 0.0228
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 41s 194ms/step - loss: 9.8603e-06 - mae: 0.0020 - val_loss: 3.4695e-04 - val_mae: 0.0123
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 41s 195ms/step - loss: 1.1834e-05 - mae: 0.0022 - val_loss: 7.0997e-04 - val_mae: 0.0214
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 41s 193ms/step - loss: 1.0369e-05 - mae: 0.0019 -